In [4]:
import numpy as np
import torch
import sys

from scipy.optimize       import minimize
from libraries.graph      import graph_POSCAR_encoding
from torch_geometric.data import Data
from pymatgen.core        import Structure

# Graph embedding

In [5]:
distance_threshold = 5
y = 1

structure = Structure.from_file('POSCAR')

nodes, edges, attributes = graph_POSCAR_encoding(structure,
                                                 distance_threshold=distance_threshold,
                                                 encoding_type='sphere-images')
temp = Data(x=nodes,
            edge_index=edges.t().contiguous(),
            edge_attr=attributes.flatten(),
            y=torch.tensor([y], dtype=torch.float)
           )
temp

Data(x=[32, 4], edge_index=[2, 313], edge_attr=[313], y=[1])

In [6]:
edges = temp.edge_index.detach().cpu().numpy().T
weights = temp.edge_attr.detach().cpu().numpy()

(array([[ 0,  1],
        [ 0,  2],
        [ 0,  3],
        [ 0,  4],
        [ 0,  5],
        [ 0,  6],
        [ 0,  7],
        [ 0,  8],
        [ 0, 16],
        [ 0, 18],
        [ 0, 19],
        [ 0, 20],
        [ 0, 21],
        [ 1,  2],
        [ 1,  3],
        [ 1,  4],
        [ 1,  5],
        [ 1,  6],
        [ 1,  7],
        [ 1,  8],
        [ 1,  9],
        [ 1, 13],
        [ 1, 14],
        [ 1, 16],
        [ 1, 17],
        [ 1, 18],
        [ 1, 19],
        [ 1, 20],
        [ 1, 21],
        [ 1, 29],
        [ 1, 30],
        [ 2,  3],
        [ 2,  4],
        [ 2,  5],
        [ 2,  6],
        [ 2,  7],
        [ 2,  8],
        [ 2, 14],
        [ 2, 16],
        [ 2, 17],
        [ 2, 18],
        [ 2, 19],
        [ 2, 20],
        [ 2, 21],
        [ 2, 29],
        [ 2, 30],
        [ 3,  4],
        [ 3,  5],
        [ 3,  6],
        [ 3,  7],
        [ 3,  8],
        [ 3, 14],
        [ 3, 15],
        [ 3, 16],
        [ 3, 17],
        [ 

In [7]:
coordinates = []
for s in structure:
    coordinates.append(s.coords)
coordinates = np.array(coordinates)

array([[-2.73763334, -2.58473027, -0.76132067],
       [-2.5707281 , -1.28468464, -0.16361727],
       [-3.48011322, -0.63559588,  0.62904367],
       [-3.02931469,  0.53824089,  1.02520646],
       [-1.79251228,  0.63361609,  0.4614824 ],
       [-1.48325346, -0.46761229, -0.27656826],
       [-0.24998085, -0.61943095, -0.9648459 ],
       [ 0.03964307, -1.63331325, -1.59586897],
       [ 0.58561689,  0.4978124 , -0.84759094],
       [ 1.88148823,  0.43340383, -1.52203544],
       [ 3.0853184 ,  0.04769889, -0.63236479],
       [ 3.5860367 ,  1.18039299,  0.08893131],
       [ 2.85732827, -1.11722621,  0.33805208],
       [ 1.86951972, -0.8014892 ,  1.3178246 ],
       [ 0.33169336,  1.60943338, -0.02206475],
       [ 1.19281926,  2.47949682,  0.1476206 ],
       [-0.91516249,  1.68124318,  0.59772555],
       [-1.25008219,  2.80559858,  1.45133247],
       [-3.7198818 , -2.98453241, -0.49657246],
       [-2.66279041, -2.48142567, -1.84668035],
       [-1.95573804, -3.24604995, -0.379

In [ ]:
mw = np.max(weights)
weights /= mw

In [19]:
# Function to calculate the squared difference between distances and weights
def objective(positions, edges, weights):
    positions = positions.reshape(-1, 3)  # Reshape to 2D array
    errors = 0
    for edge, weight in zip(edges, weights):
        p1 = positions[edge[0]]
        p2 = positions[edge[1]]
        distance = np.linalg.norm(p2 - p1)
        errors += np.power(distance - weight, 2)
    print(errors)
    return errors

# Initial guess for the positions
#initial_positions = np.random.rand(len(edges) * 3)  # Initialize all points at origin, 1D array
initial_positions = coordinates.reshape(-1, 1).ravel()

solution = minimize(objective, initial_positions, args=(edges, weights),
                    method='Powell')

# Check convergence status
if solution.success:
    print("Converged to a solution.")
else:
    print("Failed to converge:", solution.message)

2.780229430544535e-12
2.780229430544535e-12
3.7849773227696755
12.32820381895744
1.7327557150492683
0.5890333840098652
0.0007776028594760309
4.848885898311676e-07
3.147821912194067e-12
2.7801842178207763e-12
2.780184222969334e-12
2.7801842279557825e-12
2.7801842178207763e-12
5.658754540296612
19.064446982848228
2.469285308542655
0.8850219986485809
0.00034523106028413287
5.294475916572116e-08
2.7803788756054628e-12
2.7779449918265066e-12
2.777945239946864e-12
2.7779452402461483e-12
2.7779449918265066e-12
1.8376707263700154
9.60989870047029
1.2736789141639338
0.3121501145641922
0.005972947263444204
5.572305918026786e-05
7.749949364007815e-09
2.7873329431725474e-12
2.7777361920616576e-12
2.7777362177761058e-12
2.7777362182035602e-12
2.7777361920616576e-12
6.9014113972569575
20.753955782657968
2.8653692718324795
1.048649418321418
0.00018452923449924536
1.5163945036227717e-08
2.7458884781907863e-12
2.7457141688413855e-12
2.7457174681486963e-12
2.7457174694345e-12
2.7457141688413855e-12
7.81

KeyboardInterrupt: 

In [20]:
from libraries.graph import find_closest_key, composition_concentration_from_keys

In [24]:
file_name='POSCAR'
POSCAR_name = None

# Get name for the first line of the POSCAR
POSCAR_name = POSCAR_name or 'POSCAR from GenerativeModels'

# Clone the input graph to preserve the original structure
new_graph = temp.clone()

# Load and detach embeddings for the graph nodes
data_embeddings = new_graph.x.detach().cpu().numpy()

# Loading dictionary of available embeddings for atoms
available_embeddings = {}
with open('../VASP/atomic_masses.dat', 'r') as atomic_masses_file:
    for line in atomic_masses_file:
        key, mass, charge, electronegativity, ionization_energy = line.split()

        # Check if all information is present
        if all(val != 'None' for val in (mass, charge, electronegativity, ionization_energy)):
            available_embeddings[key] = np.array([mass, charge, electronegativity, ionization_energy], dtype=float)

# Get most similar atoms for each graph node and create a list of keys
keys = [find_closest_key(available_embeddings, emb) for emb in data_embeddings]

# Get the position of each atom in direct coordinates
#direct_positions = graph_to_cartesian_positions(graph)
#cartesian_positions = solution.x.reshape(-1, 3)*mw
#cartesian_positions = solution.x.reshape(-1, 3)
cartesian_positions = coordinates

# Get elements' composition, concentration, and positions
POSCAR_composition, POSCAR_concentration, POSCAR_positions = composition_concentration_from_keys(keys, cartesian_positions)

In [25]:
lattice_vectors = np.array([[10, 0,   0],
                            [0,   10, 0],
                            [0,   0,   10]])

# Write file
with open('CONTCAR', 'w') as POSCAR_file:
    # Delete previous data in the file
    POSCAR_file.truncate()
    
    # Write POSCAR's name
    POSCAR_file.write(f'{POSCAR_name}\n')

    # Write scaling factor (assumed to be 1.0)
    POSCAR_file.write('1.0\n')

    # Write lattice parameters (assumed to be orthogonal)
    np.savetxt(POSCAR_file, lattice_vectors, delimiter=' ')

    # Write composition (each different species, previously sorted)
    np.savetxt(POSCAR_file, [POSCAR_composition], fmt='%s', delimiter=' ')

    # Write concentration (number of each of the previous elements)
    np.savetxt(POSCAR_file, [POSCAR_concentration], fmt='%d', delimiter=' ')

    # Write position in cartesian form
    POSCAR_file.write('Cartesian\n')
    np.savetxt(POSCAR_file, POSCAR_positions, delimiter=' ')